In [1]:
from aimon import Client, DatasetRecord

# Create an AIMon client

This creates the AIMon client that will be used for the various different operations under evaluation and continuous monitoring of production applications.

In [2]:
# Create the AIMon client. You would need an API Key (that can be retrieved from the UI in your user profile). 
# Your user email is also needed to track ownership of components such as datasets and applications.
aimon_client = Client(api_key="cfc819b4d923f5d86fb344655efac1cea0f0a247819716ad50c9b837fd188683", email="preetam@aimon.ai")

# Generative AI Model

A model is a generative model that will be powering your application. 

In [3]:
# Pick from existing model model types in the company. These are created by you or other member of your organization.
# The AIMon client has a convenience function to easily retrieve this.
list_model_types = aimon_client.list_model_types()
print(list_model_types)

['GPT-4', 'Llama-2']


In [4]:
# Using the AIMon client, create or get a model for a given model type. 
# This API will automatically create a new model if it does not exist.
my_model = aimon_client.model(
    "my_gpt4_model_fine_tuned", 
    model_type="GPT-4", 
    description="This model is a GPT4 based model and is fine tuned on the awesome_finetuning dataset", 
    metadata={"model_s3_location":"s3://bucket/key"}
)

# LLM application

This is to create or get an application that is using the above model. Each application is versioned i.e., each application is associated with a particular model for a given version of the application. When you use a different model for the same application, AIMon will automatically increment the version of the application. 

In [5]:
from aimon import ApplicationStage
# Using the AIMon client, create or get an existing application
new_app = aimon_client.application(
    "my_llm_summarization_app", 
    my_model, 
    stage=ApplicationStage.EVALUATION, 
    app_type="summarization", 
    metadata={"applicaiton_url": "https://acme.com/summarization"}
)

Exception: Error, bad response: <Response [500]>

### Core LLM Application code

The below example uses Langchain to do summarization of documents using OpenAI.

In [8]:
# Lanchain app example
from langchain.text_splitter import CharacterTextSplitter
from langchain.docstore.document import Document
from langchain.llms.openai import OpenAI
from langchain.chains.summarize import load_summarize_chain

openai_api_key = "YOUR API KEY HERE"
def run_application(source_text, prompt=None, user_query=None, eval_run=None):
    # Split the source text
    text_splitter = CharacterTextSplitter()
    texts = text_splitter.split_text(source_text)
    
    # Create Document objects for the texts
    docs = [Document(page_content=t) for t in texts[:3]]
    
    # Initialize the OpenAI module, load and run the summarize chain
    llm = OpenAI(temperature=0, openai_api_key=openai_api_key)
    chain = load_summarize_chain(llm, chain_type="map_reduce")
    summary = chain.run(docs)
    #print("Generated Summary: {}".format(summary))

    # Analyze quality of the generated output using AIMon
    dr = DatasetRecord(prompt=prompt, user_query=user_query, context_docs=[d.page_content for d in docs])
    res = aimon_client.analyze(new_app, dr, summary, eval_run)
    print("Aimon response: {}\n".format(res))

# Evaluation of the LLM Application

Before deploying the application to production, it is a good idea to test it end to end with either a curated golden dataset or a snapshot of production traffic. In this section, we will demonstrate how AIMon can assist you to perform these tests.

### Evaluation Dataset

The dataset should be a CSV file with these columns: 
 * "prompt": This is the prompt used for the LLM
 * "user_query": This the query specified by the user 
* "context_docs": These are context documents that are either retrieved from a RAG or through other methods. 
                  For tasks like summarization, these documents could be directly specified by the user.

In [9]:
# Create a new dataset
# dataset1 = aimon_client.create_dataset("test_evaluation_dataset_1.csv", "./aimon_sdk_files/test_evaluation_dataset_1.csv", "This is one custom dataset")
# dataset2 = aimon_client.create_dataset("test_evaluation_dataset_2.csv", "./aimon_sdk_files/test_evaluation_dataset_2.csv", "This is another custom dataset")

In [10]:
dataset1 = aimon_client.get_dataset("test_evaluation_dataset_1.csv")
dataset2 = aimon_client.get_dataset("test_evaluation_dataset_2.csv")

### Dataset Collection

You can define a collection of evaluation datasets for ease of use. 

In [11]:
# Create a new dataset collection
dataset_collection = aimon_client.dataset_collection(
    "my_first_dataset_collection", 
    [dataset1, dataset2], 
    "This is a collection of two datasets."
)

### Evaluation

An evaluation is associated with a dataset collection and an application.

In [12]:
# Using the AIMon client, create a new evaluation
evaluation = aimon_client.evaluation("offline_evaluation_fine_tuned_model", new_app, dataset_collection)

### New Run
A "run" is an instance of an evaluation that you would like to track metrics against. You could have multiple runs of the same evaluation. This is typically done is a CI/CD context where the same evaluation would run at regular intervals. Since LLMs are probabilitic in nature, they could produce different outputs for the same query and context. It is a good idea to run the evaluations regularly to understand the variations of outputs produced by your LLMs. In addition, runs give you the ability to choose different metrics for each run. 

Metrics are specified using the `metrics_config` parameter in the format shown below. The keys indicate the type of metric computed and the values are the specific algorithms used to compute those metrics. For most cases, we recommend using the `default` algorithm.

Tags allow you to specify metadata like the application commit SHA or other key-value pairs that you want to insert for analytics purposes.

In [13]:
# Using the AIMon client, create a new evaluation run. 
eval_run = aimon_client.new_run(
    evaluation, 
    metrics_config={'hallucination': 'default', 'toxicity': 'default', 'conciseness': 'default', 'completeness': 'default'},
    tags={'application_commit_sha': '09395f57aac0769c55a0b3145b3f83e8ced148ef'}
)

In [14]:
# Run the application on all the records in the dataset collection. 
for record in dataset_collection.records():
    # Run the application code
    run_application(record.context_docs, record.prompt, record.user_query, eval_run)

# You can view metrics for your application on the UI: https://www.app.aimon.ai/llmapps

/Users/preetamjoshi/projects/aimon/aimon/datascience/aimon_dev/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.llms.openai.OpenAI` was deprecated in langchain-community 0.0.10 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAI`.
  warn_deprecated(
/Users/preetamjoshi/projects/aimon/aimon/datascience/aimon_dev/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `run` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(


Aimon response: {'message': 'Data successfully sent to AIMon.'}

Aimon response: {'message': 'Data successfully sent to AIMon.'}

Aimon response: {'message': 'Data successfully sent to AIMon.'}

Aimon response: {'message': 'Data successfully sent to AIMon.'}

Aimon response: {'message': 'Data successfully sent to AIMon.'}

Aimon response: {'message': 'Data successfully sent to AIMon.'}

Aimon response: {'message': 'Data successfully sent to AIMon.'}

Aimon response: {'message': 'Data successfully sent to AIMon.'}



# Production

Once you have built enough confidence through your evaluations of your application, you can deploy it to production. AIMon gives you the ability to continuously monitor your application for the configured metrics in production.

In [12]:
source_text = """
Large Language Models (LLMs) have become integral to automating and enhancing various business processes. 
However, a significant challenge these models face is the concept of \"hallucinations\" - outputs that, 
although fluent and confident, are factually incorrect or nonsensical. For enterprises relying on AI 
for decision-making, content creation, or customer service, these hallucinations can undermine credibility, 
spread misinformation, and disrupt operations. Recently, AirCanada lost a court case due to hallucinations 
in its chatbot [7]. Also, the 2024 Edelman Trust Barometer reported a drop in trust in AI companies from 
61% to 53% compared to 90% 8 years ago [8]. Recognizing the urgency of the issue, we have developed a 
state-of-the-art system designed for both offline and online detection of hallucinations, ensuring higher 
reliability and trustworthiness in LLM outputs.
"""

In [13]:
run_application(source_text, prompt="Langhchain based summarization of documents")

Aimon response: {'message': 'Data successfully sent to AIMon.'}

